In [1]:
!pip3 install mediapipe

     ---------------------------------------- 49.8/49.8 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 42.5/42.5 MB 5.5 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2

In [18]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [19]:
alphabets = list("ABCDEFGHIKLMNOPQRSTUVWXY")
print(alphabets)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [27]:
temp_dataset = []

for label in alphabets:
    dataset_directory_prefix = '/ProjetIA/Raw_data/data'
    for dirname, _, filenames in os.walk(dataset_directory_prefix + label):
        for filename in filenames:
            pathname = os.path.join(dirname, filename)

            with mp_hands.Hands(
                static_image_mode=True,
                max_num_hands=1,
                min_detection_confidence=0.3
            ) as hands:

                image = cv2.flip(cv2.imread(pathname), 1)
                result = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                landmark_vertices_xyz = []
                if not result.multi_hand_landmarks:
                    continue

                for l in result.multi_hand_landmarks[0].landmark:
                    landmark_vertices_xyz.append(l.x)
                    landmark_vertices_xyz.append(l.y)
                    landmark_vertices_xyz.append(l.z)

                temp_dataset.append((*landmark_vertices_xyz, label))
                
    print(f'imported: {label}')
        
landmark_vertices_xyz_label = []
for idx in range(21):
    for char in list('xyz'):
        vertex_label = char+str(idx)
        landmark_vertices_xyz_label.append(vertex_label)
            
print(*landmark_vertices_xyz_label)
        
dataset = pd.DataFrame(temp_dataset, columns=[*landmark_vertices_xyz_label, 'target'])
dataset.head()

imported: A
imported: B
imported: C
imported: D
imported: E
imported: F
imported: G
imported: H
imported: I
imported: K
imported: L
imported: M
imported: N
imported: O
imported: P
imported: Q
imported: R
imported: S
imported: T
imported: U
imported: V
imported: W
imported: X
imported: Y
x0 y0 z0 x1 y1 z1 x2 y2 z2 x3 y3 z3 x4 y4 z4 x5 y5 z5 x6 y6 z6 x7 y7 z7 x8 y8 z8 x9 y9 z9 x10 y10 z10 x11 y11 z11 x12 y12 z12 x13 y13 z13 x14 y14 z14 x15 y15 z15 x16 y16 z16 x17 y17 z17 x18 y18 z18 x19 y19 z19 x20 y20 z20


,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,target


In [15]:
dataset.to_csv('data.csv', index=False)

In [16]:
from sklearn.model_selection import train_test_split

X = dataset.drop(['target'],axis=1).values
y = dataset['target'].values

# Choose your test size to split between training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.